In [1]:
import requests
from bs4 import BeautifulSoup
import os
from astropy.io import fits
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
from tqdm import tqdm
import time
from astropy.utils.data import get_pkg_data_filename
from astropy.convolution import convolve, Gaussian1DKernel

### Obtenemos los enlaces de cada archivo y creamos un archivo de texto con ellos

In [ ]:
base_url = "https://data.desi.lbl.gov/public/edr/spectro/redux/fuji/healpix/sv3/dark/"

# Obtengo las carpetas y archivos FITS desde la página HTML
def get_folders_and_fits(url):
    response = requests.get(url)
    # Verifica si la respuesta falló y muestra un mensaje
    if response.status_code != 200:
        print("Verifica tu conexión a Internet para asegurarte de que esté funcionando correctamente.")
        return [], []
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    folders = []
    fits_files = []

    # Busco todos los enlaces en la página
    for link in soup.find_all('a', href=True):
        href = link['href']
        text = link.text
        if href.endswith('/') and not text.startswith('..') and not href.endswith('logs/'):
            # Es una carpeta
            folder_name = text.rstrip('/')
            folders.append(folder_name)
        elif text.startswith('redrock-sv3-dark-') or text.startswith('coadd-sv3-dark-'):
            # Es un archivo FITS
            fits_files.append(text)
    
    return folders, fits_files

# Explorar los archivos de la ultima, tercera, capa. Aqui estan los .fits
def explore_third_layer_folders(url):
    _ , fits_files = get_folders_and_fits(url)
    nstype=[]
    # FITS coadd y redrock
    for fits_file in tqdm(fits_files):
        with open('urlspectra.txt', 'a') as archivo:
            if fits_file.startswith('redrock-sv3-dark-'): 
                red_rock_fits_urls = url + fits_file  #aqui obtengo todos los urls que me llevan a todos los fits de redrock
                archivo.write(red_rock_fits_urls+"\n")
            if fits_file.startswith('coadd-sv3-dark-'): 
                coadd_fits_urls = url + fits_file
                archivo.write(coadd_fits_urls+"\n")
        print("copiado")
            

    
# Explorar las carpetas de la segunda capa 10016/ 10145/ etc/
def explore_second_layer_folders(url):
    folders, _ = get_folders_and_fits(url)
    for folder in folders:
        folder_url = url + folder + '/'
        explore_third_layer_folders(folder_url)                
                
# Explora las carpetas de la primera capa: 100/ 101/ etc/
def explore_first_layer_folders(url):
    folders, _ = get_folders_and_fits(url)
    for folder in folders:
        folder_url = url + folder + '/'
        explore_second_layer_folders(folder_url)
        
        
# Comienzo la exploración desde la URL base en la primera capa
explore_first_layer_folders(base_url)

### Leemos el archivo de texto con los enlaces, bajamos los espectros y los almacenamos en nuevos conjuntos de archivos .fits

In [10]:
with open('urlspectra.txt', 'r') as archivo:
    n=0
    STYPE=[]
    Z=[]
    TARGET=[]
    BFLUX=[]
    RFLUX=[]
    ZFLUX=[]
    lineas = archivo.readlines()  # Lee todas las líneas del archivo y las almacena en una lista
    for i in range(720,len(lineas)-2, 2): #Este rango va de el primero al tercer fits es decir los primeros 20 fits con 10 readrock y 10
        # Imprime dos líneas a la vez (par de líneas)
        stype=[]
        z=[]
        target=[]
        Bflux=[]
        Rflux=[]
        Zflux=[]
        
        if i + 1 < len(lineas):
            coadd=fits.open(lineas[i])
            readrock=fits.open(lineas[i + 1])
            read_readrock=Table.read(readrock, hdu=1)
            read_coadd = Table.read(coadd, hdu=1) 
            targetID_coadd=read_coadd['TARGETID'].data #Target ID lista # Redshift lista
            spectype=read_readrock['SPECTYPE'].data    # Tipo espectral lista
            Z_r=read_readrock['Z'].data # 
            for j in range(len(targetID_coadd)): #elimina los targetID negativos
                if targetID_coadd[j]>=0:
                    stype.append(spectype[j])
                    z.append(Z_r[j])
                    target.append(targetID_coadd[j])
                    Bflux.append(coadd[4].data[j])
                    Rflux.append(coadd[9].data[j])
                    Zflux.append(coadd[14].data[j])     
            #une los espectros de cada archivo fits
            TARGET+=target
            Z+=z
            STYPE+=stype
            BFLUX+= Bflux
            RFLUX+= Rflux
            ZFLUX+= Zflux
            n+=1
            print("Copiado: "+str(n))
            
    # Define los tipos de datos
    dtype = [('SPECTYPE', 'S7'), ('Z', 'float'), ('TARGETID', 'float')]
    # Crea un arreglo estructurado de NumPy
    data = np.zeros(len(Z), dtype=dtype)
    data['SPECTYPE'] = STYPE
    data['Z'] = Z
    data['TARGETID'] = TARGET
    # Crea una tabla FITS usando fits.BinTableHDU
    hdu = fits.BinTableHDU.from_columns(data)
    # Crea los arreglos para 
    data_Bflux = np.array(BFLUX, dtype=np.float32)
    data_Rflux = np.array(RFLUX, dtype=np.float32)
    data_Zflux = np.array(ZFLUX, dtype=np.float32)
    # Crea un objeto ImageHDU
    hdub = fits.ImageHDU(data_Bflux)
    hdur = fits.ImageHDU(data_Rflux)
    hduz = fits.ImageHDU(data_Zflux)
    # Establece el nombre del HDU como 'B_FLUX'
    hdub.name = 'B_FLUX'
    hdur.name = 'R_FLUX'
    hduz.name = 'Z_FLUX'
    #Agrega al fits.
    nombre_archivo = 'DataDESI.fits'
    hdulist = fits.open(nombre_archivo, mode='append')
    hdulist.append(hdu)
    hdulist.append(hdub)
    hdulist.append(hdur)
    hdulist.append(hduz)
    hdulist.close() 


Copiado: 1
Copiado: 2
Copiado: 3
Copiado: 4
Copiado: 5
Copiado: 6
Copiado: 7
Copiado: 8
Copiado: 9
Copiado: 10
Copiado: 11
Copiado: 12
Copiado: 13
Copiado: 14
Copiado: 15
Copiado: 16
Copiado: 17
Copiado: 18


### Este es el nombre de las carpetas que condensan todos los datos originales de la base de datos de DESI

In [ ]:
['DataDESI_1_36.fits', 'DataDESI_37_72.fits', 'DataDESI_73_108.fits', 'DataDESI_109_144.fits', 'DataDESI_145_180.fits', 'DataDESI_181_216.fits'
, 'DataDESI_217_252.fits', 'DataDESI_253_288.fits', 'DataDESI_289_324.fits', 'DataDESI_325_360.fits', 'DataDESI_361_379.fits']

### Exploracion de los datos

In [2]:
from astropy.table import Table
espc = fits.open('DataDESI_361_379.fits')
espc.info()
print(len(espc[2].data))

Filename: DataDESI_361_379.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1                1 BinTableHDU     14   21772R x 3C   [7A, D, D]   
  2  B_FLUX        1 ImageHDU         8   (2751, 21772)   float32   
  3  R_FLUX        1 ImageHDU         8   (2326, 21772)   float32   
  4  Z_FLUX        1 ImageHDU         8   (2881, 21772)   float32   
21772


In [57]:
conteo_a = np.sum(Table.read(espc, hdu=1)["SPECTYPE"].data == 'GALAXY')

print("El elemento 'a' aparece", conteo_a, "veces en el NumPy array.")

El elemento 'a' aparece 19364 veces en el NumPy array.


In [58]:
conteo_a = np.sum(Table.read(espc, hdu=1)["SPECTYPE"].data == 'STAR')

print("El elemento 'a' aparece", conteo_a, "veces en el NumPy array.")

El elemento 'a' aparece 970 veces en el NumPy array.


In [59]:
conteo_a = np.sum(Table.read(espc, hdu=1)["SPECTYPE"].data == 'QSO')

print("El elemento 'a' aparece", conteo_a, "veces en el NumPy array.")

El elemento 'a' aparece 1438 veces en el NumPy array.


In [8]:
print(Table.read(espc, hdu=1))

SPECTYPE           Z                   TARGETID       
-------- ---------------------- ----------------------
  GALAXY     0.8096589134393345  3.963312758430371e+16
  GALAXY     0.9282989193404377  3.963313202187755e+16
  GALAXY     1.5251344667187916  3.963313643848008e+16
  GALAXY    0.25679214910795595  3.963313202187697e+16
  GALAXY     0.7563224362494237  3.963312758430276e+16
    STAR -0.0011996796199236536 3.9633136442672536e+16
     QSO     1.1807410029513217  3.963313202187771e+16
     QSO     0.9350379047671412 3.9633127584302856e+16
  GALAXY     0.8423552934538804 3.9633127584303496e+16
  GALAXY     0.9259450310404335   3.96331320218769e+16
     ...                    ...                    ...
  GALAXY     0.9553647756528504  3.963315407972351e+16
  GALAXY     0.7574984894419702  3.963315407972291e+16
  GALAXY     1.4918708566290344  6.160939107074703e+17
  GALAXY     0.4213718959427256 3.9633158404051016e+16
     QSO     1.1677343519034808   3.96331540839138e+16
    STAR -

In [13]:
Table.read(espc, hdu=1)['Z'].data

array([ 0.80965891,  0.92829892,  1.52513447, ...,  0.36303014,
       -0.00199569,  0.69072692])

In [65]:
print(len(espc[2].data))
print(len(espc[4].data[0]))

21772
2881
